<a href="https://colab.research.google.com/github/i-r-a/CSE_574/blob/master/Larger_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Subset containing Balloon Flower, Pink Primrose, Common Dandelion, King Protea, Bird of Paradise


In [0]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image

from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import io
labels = pd.read_csv('drive/My Drive/fleur/fleur_set.csv')

In [4]:
labels.tail()

,file,species
630,982.jpg,Pink_Primrose
631,9859.jpg,Pink_Primrose
632,9896.jpg,Pink_Primrose
633,9913.jpg,Pink_Primrose
634,9963.jpg,Pink_Primrose


In [0]:
def get_image(row_id, root="drive/My Drive/fleur/"):
    """
    Converts an image number into the file path where the image is located, 
    opens the image, and returns the image as a numpy array.
    """
    filename = "{}".format(row_id)
    file_path = os.path.join(root, filename)
    img = Image.open(file_path)
    return np.array(img)

In [0]:
def create_features(img):
    # flatten three channel color image
    color_features = img.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img)
    # get HOG features from greyscale image
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    # combine color and hog features into a single array
    flat_features = np.hstack(color_features)
    return flat_features

In [0]:
def create_feature_matrix(label_dataframe):
    features_list = []
    
    for img_id in label_dataframe.file:
        # load image
        
        img = get_image(img_id)
        # get features for image
        image_features = create_features(img)
        features_list.append(image_features)
    # convert list of arrays into a matrix
    feature_matrix = np.array(features_list)
    return feature_matrix



In [0]:
# run create_feature_matrix on our dataframe of images
feature_matrix = create_feature_matrix(labels)

In [0]:
ss = StandardScaler()
# run this on our feature matrix
fleur_stand = ss.fit_transform(feature_matrix)

pca = PCA(n_components=500)
# use fit_transform to run PCA on our standardized matrix
fleur_pca = ss.fit_transform(fleur_stand)

In [10]:
X = pd.DataFrame(fleur_pca)
y = pd.Series(labels.species.values)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.3,
                                                    random_state=42)

# look at the distrubution of labels in the train set
pd.Series(y_train).value_counts()

Pink_Primrose       160
Common_Dandelion    121
Bird_of_paradise     73
Balloon_Flower       67
King_Protea          23
dtype: int64

In [11]:
svm = SVC(kernel='linear', C= float('inf'),probability=True)

svm.fit(X_train, y_train)

SVC(C=inf, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [12]:
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy is: ', accuracy)

Model accuracy is:  0.8429319371727748
